In [1]:
import numpy as np 
import pandas as pd
from wordcloud import WordCloud
from wordcloud import STOPWORDS
import re
from sklearn.externals
import joblib

from bs4 import BeautifulSoup
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import warnings
import math
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input,Dense,LSTM
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Conv1D, Conv2D
from keras.layers import MaxPooling1D, GlobalMaxPooling1D ,MaxPooling2D
from keras.layers import Dense, Input , Dropout, Flatten
import h5py
from tensorflow.python.keras.callbacks import TensorBoard,ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
from time import time
import pickle 
from numpy import zeros
from tensorflow.keras.layers import (
    BatchNormalization)
from keras.layers import Bidirectional

C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
def decontracted(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\'I'm", "I am", phrase)
    phrase = re.sub(r"I\'ve", "I have", phrase)
    phrase = re.sub(r"isn\'t", "is not", phrase)
    phrase = re.sub(r"we\'ll", "we will", phrase)
    phrase = re.sub(r"we\'re", "we are", phrase)
    phrase = re.sub(r"we\'ve", "we have", phrase)
    phrase = re.sub(r"weren\'t", "were not", phrase)
    phrase = re.sub(r"quake", "earthquake", phrase)
    phrase = re.sub(r"nado", "tornado", phrase)
    return phrase

In [3]:
def clean_tweet(sentence):
    sentence = re.sub(r"http\S+", "", sentence)
    sentence = BeautifulSoup(sentence, 'lxml').get_text()
    sentence = decontracted(sentence)
    sentence = re.sub("\S*\d\S*", "", sentence).strip()
    sentence = re.sub('[^A-Za-z]+', ' ', sentence)
    sentence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in stopwords)
    return sentence.strip()

In [4]:
df_train = pd.read_csv("train.csv")

In [5]:
stopwords = set(STOPWORDS)

In [6]:
preprocessed_tweets = []
for sentence in df_train['text'].values:
    preprocessed_tweets.append(clean_tweet(sentence))

C:\Users\ADMIN\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [7]:
Y = df_train['target'].values

In [84]:
tokenizer_tweet = Tokenizer()
tokenizer_tweet.fit_on_texts(preprocessed_tweets)
joblib.dump(tokenizer_tweet, 'token_tweet.pkl')    

['token_tweet.pkl']

In [85]:
tweet_tokenlen = len(tokenizer_tweet.word_index)+1
tweet_token_text  = tokenizer_tweet.texts_to_sequences(preprocessed_tweets)
X_pad = pad_sequences(tweet_token_text, maxlen=35 , padding='post' )
X_train, X_test, y_train, y_test = train_test_split(X_pad, Y, stratify=df_train['target'], test_size=0.20)
embedding_matrix = zeros((tweet_tokenlen, 100))

In [86]:
glove_model = {}
f = open('glove.6B.100d.txt',encoding="utf8")
for line in f:
    split_line  = line.split()
    word = split_line[0]
    embedding  = np.asarray(split_line [1:], dtype='float32')
    glove_model [word] = embedding 
f.close()

In [87]:
glove_keys = glove_model.keys()
for word, i in tokenizer_tweet.word_index.items():
    if word in glove_keys:
        embedding_vector = glove_model.get(word)
        embedding_matrix[i] = embedding_vector


In [88]:
embedding_matrix.shape

(15615, 100)

In [89]:
np.save('embeddings.npy', embedding_matrix)

In [90]:
all_embeddings = np.load('embeddings.npy')

In [91]:
filepath="weightsflask1.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy',save_best_only=True, mode='max', verbose=1)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/model_flask".format(time()))
callbacks_1 = [checkpoint,tensorboard_callback]

In [1]:
embedding_vecor_length = 32
model7 = Sequential()
model7.add(Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1], weights = [embedding_matrix], input_length=35))
model7.add(LSTM(108,return_sequences=True))
model7.add(Dropout(0.5))
model7.add(LSTM(64,return_sequences=True))
model7.add(Dropout(0.5))
model7.add(LSTM(32))
model7.add(Dropout(0.3))
model7.add(Dense(1, activation='relu'))
print(model7.summary())

NameError: name 'Sequential' is not defined

In [93]:
model7.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
history_7 = model7.fit(X_train,
          y_train,
          batch_size=64,
          validation_data=(X_test, y_test),
          epochs=15,callbacks=[callbacks_1])

Epoch 1/15
96/96 [==============================] - ETA: 0s - loss: 0.6827 - accuracy: 0.7379
Epoch 00001: val_accuracy improved from -inf to 0.72095, saving model to weightsflask1.hdf5
96/96 [==============================] - 27s 210ms/step - loss: 0.6827 - accuracy: 0.7379 - val_loss: 0.6629 - val_accuracy: 0.7209
Epoch 2/15
96/96 [==============================] - ETA: 0s - loss: 0.5653 - accuracy: 0.7732
Epoch 00002: val_accuracy improved from 0.72095 to 0.80368, saving model to weightsflask1.hdf5
96/96 [==============================] - 19s 196ms/step - loss: 0.5653 - accuracy: 0.7732 - val_loss: 0.4940 - val_accuracy: 0.8037
Epoch 3/15
96/96 [==============================] - ETA: 0s - loss: 0.5099 - accuracy: 0.8153
Epoch 00003: val_accuracy did not improve from 0.80368
96/96 [==============================] - 19s 194ms/step - loss: 0.5099 - accuracy: 0.8153 - val_loss: 0.9121 - val_accuracy: 0.7912
Epoch 4/15
96/96 [==============================] - ETA: 0s - loss: 0.5582 - acc

In [94]:
model7.save('model_flask')

INFO:tensorflow:Assets written to: model_flask\assets


INFO:tensorflow:Assets written to: model_flask\assets


In [11]:
import keras

## Here is where I am unable to predict properly, even thought my function and process is correct from previous Direct model.

In [ ]:
def predict_tweet(string):
    model_flask = keras.models.load_model('model_flask')
    tokenizer_tweet = joblib.load('token_tweet.pkl')
    tweet_tokenlen = len(tokenizer_tweet.word_index)+1
    tweet = clean_tweet(string)
    tweet_token_text  = tokenizer_tweet.texts_to_sequences(tweet)
    tweet_token_text = pad_sequences(tweet_token_text, maxlen=35 , padding='post' )
    #embedding_matrix = np.load('embeddings.npy')
    pred = model_flask.predict_classes(tweet_token_text)
    #pred =np.argmax(pred)

    print(pred)
    if pred == 0:
        prediction = "Hoax"
    else:
        prediction = "Real Disaster"
    return prediction

## Here I get the no attribute error

In [23]:
print(predict_tweet('Forest fire near La Ronge Sask. Canada'))

AttributeError: 'Sequential' object has no attribute 'predict_classes'

## I go through stack over flow and find this solution

In [122]:
def predict_tweet(string):
    model_flask = keras.models.load_model('model_flask')
    tokenizer_tweet = joblib.load('token_tweet.pkl')
    tweet = clean_tweet(string)
    #print(tweet)
    tweet=  [tweet]
    tweet_token_text  = tokenizer_tweet.texts_to_sequences(tweet)
    #print(tweet_token_text)
    #print(len(tweet_token_text))
    tweet_token_text = pad_sequences(tweet_token_text, maxlen=35 , padding='post')
    #print(tweet_token_text.shape)
    #embedding_matrix = np.load('embeddings.npy')
    pred = model_flask.predict(tweet_token_text)
    #pred =np.argmax(pred,axis=1)
    #predictions = (model_flask.predict(tweet_token_text) > 0.5).astype("int32")
    pred = np.array(list(map(lambda x : 'Real Disaster' if x > 0.5 else 'Hoax',pred)))
    #pred = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',pred)))
    print(pred)
    print(pred.shape)
    print(type(pred))


In [36]:
def predict_tweet(string):
    model_flask = keras.models.load_model('model_flask')
    tokenizer_tweet = joblib.load('token_tweet.pkl')
    tweet = clean_tweet(string)
    tweet=  [tweet]
    tweet_token_text  = tokenizer_tweet.texts_to_sequences(tweet)
    tweet_token_text = pad_sequences(tweet_token_text, maxlen=35 , padding='post')
    #pred = model_flask.predict(tweet_token_text)
    pred = (model_flask.predict(tweet_token_text) > 0.5).astype("int32")
    if pred == 0:
        return "It's a Hoax"
    else:
        return "Real disaster!!"
    #pred = (list(map(lambda x : 'Real Disaster' if x > 0.5 else 'Hoax',pred)))
    #return pred

In [111]:
a = 'Forest fire near La Ronge Sask. Canada'

In [103]:
type(a)

str

In [104]:
a=  a.split(',')

In [106]:
a

['Forest fire near La Ronge Sask. Canada']

In [37]:
print(predict_tweet('Forest fire near La Ronge Sask. Canada'))

1/1 [==============================] - 1s 1s/step
Real disaster!!


In [38]:
print(predict_tweet('Love my girlfriend'))

1/1 [==============================] - 1s 1s/step
It's a Hoax


In [124]:
print(predict_tweet('ablaze,Anaheim,Police: Arsonist Deliberately Set Black Church In North CarolinaåÊAblaze http://t.co/pcXarbH9An'))

ablaze anaheim police arsonist deliberately set black church north carolina ablaze
[[490, 13, 647, 6302, 159, 86, 1426, 526, 4229, 490]]
1
(1, 35)
1/1 [==============================] - 1s 1s/step
['positive']
(1,)
<class 'numpy.ndarray'>
None


In [ ]:
a.round()
# array([0., 0., 0., 1., 1., 1.])

## Again the Above error appears. Please help me solve this, I tried few more solutions from stackoverflow but none work.